In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
using_colab = True

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

PyTorch version: 2.0.1+cu118
Torchvision version: 0.15.2+cu118
CUDA is available: True
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-cq1s56c1
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-cq1s56c1
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36589 sha256=5e576a9a44f0ac78b1e7ebc358f5148f9a27458041a18f75ee0bf8d6554b6c0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-9tbfxvhl/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything
--2023-07-26 07:48:28--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.224.2.42, 13.224.2.88, 13.2

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [ ]:
import os
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from PIL import Image, ImageFilter
import random

# replace with your actual folder path
folder_path = "//content/drive/MyDrive/100/"
sky_path = "//content/drive/MyDrive/sky/"
# Get list of all .jpg files in the sky_path
sky_files = os.listdir(sky_path)
new_sky_files = [file for file in sky_files if file.endswith(".jpg")]

# Get list of all files in the directory
all_files = os.listdir(folder_path)

# Filter the list for .jpeg files
jpg_files = [file for file in all_files if file.endswith(".jpg")]


for i in range(1,len(jpg_files)):
    image_path = os.path.join(folder_path, f"{i}.jpg")

    # Open an image file
    image = cv2.imread(image_path)
    resized_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #change the pixels of images
    #new_dimensions = (512, 512)
    #resized_image = cv2.resize(image, new_dimensions, interpolation = cv2.INTER_LINEAR)

    # use segmentation anything model
    sam_checkpoint = "sam_vit_b_01ec64.pth"
    model_type = "vit_b"
    device = "cuda"
    sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
    sam.to(device=device)
    mask_generator = SamAutomaticMaskGenerator(sam)
    masks = mask_generator.generate(resized_image)

    # Assuming that 'image' is your image on which you want to draw the bounding boxes.
    image_copy = resized_image.copy()  # Copy image to avoid drawing on the original
    image_height, image_width = resized_image.shape[:2]

    # Create a copy of the image with an alpha channel
    image_with_alpha = np.dstack((image_copy, np.ones((image_height, image_width), dtype=np.uint8) * 255))

    # Loop over each mask
    sky_mask=[]
    for mask in masks:
        # Get bbox
        x, y, w, h = mask['bbox']

        if x == 0 and w >= image_width * 0.9:  # You can adjust the 0.9 depending on your specific requirements
            sky_mask = mask
            break

    # Loop over each mask
    for mask in masks:
        # Get bbox
        x, y, w, h = mask['bbox']

        if y == 0 and w >= image_width * 0.9:  # You can adjust the 0.9 depending on your specific requirements
            sky_mask = mask
            break
    if sky_mask != []:
      print(f'exist{i}')
    else:
      print('none')
    for mask in masks:
      if np.array_equal(mask, sky_mask):  # or any other condition you're looking for
          # Assuming 'mask' is a dictionary with key 'segmentation'
          sky_mask_array = np.array(mask['segmentation']).astype(np.uint8)

          # Resize the sky mask to match the image size
          sky_mask_reshaped = cv2.resize(sky_mask_array, (image_width, image_height))
          # Blurring the sky mask
          mask_blurred  = cv2.GaussianBlur(sky_mask_reshaped,(51,51),0)

          # Use the sky mask to set the alpha channel
          image_with_alpha[mask_blurred == 1, 3] = 0

          for k in range(1, image_height - 1):
              for j in range(1, image_width - 1):
                  if (image_with_alpha[k, j, 3] == 0 and image_with_alpha[k - 1, j, 3] != 0) or \
                          (image_with_alpha[k, j, 3] != 0 and image_with_alpha[k - 1, j, 3] == 0):
                      image_with_alpha[k-2:k+3, j-2:j+3] = [0, 0, 0, 255]  # Black color

          #Display the image with transparent sky region and black edge at the sky-ground boundary
          plt.imsave(f'//content/drive/MyDrive/100 new/{i}.png',image_with_alpha)
    # Randomly select a jpg image
    selected_image = random.choice(new_sky_files)
    # Load the images
    sky_image = Image.open(f'//content/drive/MyDrive/sky/{selected_image}')
    #sky_image = Image.open(f'//content/drive/MyDrive/real_dataset_change/{i}.jpg')
    main_image = Image.open('image_with_transparent_sky.png')

    # Resize the sky image to match the size of the main image, if necessary
    sky_image = sky_image.resize(main_image.size, Image.ANTIALIAS)

    # Use the alpha channel from the main image as the "mask" for the paste operation
    main_image_alpha = main_image.split()[3]

    # Paste the main image onto the sky image using the alpha channel as the mask
    sky_image.paste(main_image, (0, 0), main_image_alpha)

    # Save and display the result
    new_file_path = os.path.join("//content/drive/MyDrive/combined_dataset/", f'{i}.jpg')
    sky_image.save(new_file_path)


none
exist2
exist3
exist4
exist5
exist6
exist7
exist8
exist9
exist10
exist11
exist12
none
exist14
exist15
exist16
exist17
exist18
exist19
exist20
exist21
none
exist23
exist24
exist25
exist26
none
exist28
exist29
exist30
exist31
exist32
exist33
exist34
exist35
exist36
exist37
none
exist39
exist40
exist41
exist42
exist43
exist44
exist45
exist46
exist47
exist48
exist49
exist50
exist51
exist52
exist53
exist54
exist55
exist56
exist57
exist58
exist59
exist60
exist61
exist62
exist63
exist64
exist65
exist66
exist67
exist68
exist69
exist70
exist71
none
exist73
none
exist75
none
exist77
exist78
exist79
exist80
exist81
exist82
exist83
exist84
exist85
exist86
none
exist88
exist89
exist90
exist91
exist92
exist93
exist94
exist95
exist96
exist97
exist98
exist99
exist100
exist101
exist102
exist103
exist104
exist105
exist106
exist107
none
exist109
none
exist111
exist112
exist113
exist114
exist115
exist116
none
none
exist119
exist120
exist121
exist122
exist123
exist124
exist125
exist126
exist127
none
ex